# Improved Random Forest Model with Engineered Features

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
df = pd.read_csv('Monthly_Expanded_WholeNumbers.csv')
df['Date'] = pd.to_datetime(df[['Year','Month']].assign(DAY=1))
df = df.sort_values('Date')

In [ ]:
df['Month'] = df['Date'].dt.month
df['YearVal'] = df['Date'].dt.year
df['TimeIndex'] = np.arange(len(df))
df['Lag1'] = df['Total'].shift(1)
df['Lag12'] = df['Total'].shift(12)
df['Rolling3'] = df['Total'].rolling(3).mean()
df['Rolling12'] = df['Total'].rolling(12).mean()
df = df.dropna()

In [ ]:
features=['TimeIndex','Month','YearVal','Lag1','Lag12','Rolling3','Rolling12']
X=df[features]
y=df['Total']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,shuffle=False)
model=RandomForestRegressor(n_estimators=500,random_state=42)
model.fit(X_train,y_train)
pred=model.predict(X_test)
print('MAE:',mean_absolute_error(y_test,pred))

In [ ]:
future_rows=120
last=df.iloc[-1]
future_data=[]
for i in range(1,future_rows+1):
    e={}
    e['TimeIndex']=last['TimeIndex']+i
    e['Month']=((int(last['Month'])+i-1)%12)+1
    e['YearVal']=int(last['YearVal'])+((int(last['Month'])+i-1)//12)
    e['Lag1']=df['Total'].iloc[-1] if i==1 else future_data[-1]['Forecast']
    if i<=12: e['Lag12']=df['Total'].iloc[-12+i]
    else: e['Lag12']=future_data[-12+i]['Forecast']
    e['Rolling3']=np.mean([e['Lag1'],e['Lag12'],df['Total'].iloc[-1]])
    e['Rolling12']=df['Rolling12'].iloc[-1]
    future_data.append(e)
future_df=pd.DataFrame(future_data)
future_df['Forecast']=model.predict(future_df[features])

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['Date'],df['Total'],label='Historical')
future_dates=pd.date_range(df['Date'].iloc[-1],periods=121,freq='M')[1:]
plt.plot(future_dates,future_df['Forecast'],label='Forecast')
plt.legend()
plt.title('Improved 10-Year Forecast with Engineered Features')
plt.show()